# CSCI316 Individual Assignment 1 - Task 2

This code include:
* Preprocessing
* Binning
* Functions for Information Gain
* Implementation of Decision Tree
* Random Sampling
* View Accuracy
* Evaluation

In [110]:
import numpy as np
import pandas as pd
import warnings

## Load the dataset

In [112]:
df = pd.read_csv("data.csv", delimiter=';')

## Preprocessing

In [113]:
# Label Encoding
df['Target'] = df['Target'].map({'Graduate': 0, 'Enrolled': 1,'Dropout': 2})

In [114]:
# Binning
# Only performed on features with many continuous values (numerous unique values).
pre_qualification = df['Previous qualification (grade)']
bins = [0, 100, 150, 200]
labels = ['0-100', '101-150', '151-200']
df['Pre_qualification_group'] = pd.cut(pre_qualification, bins=bins, labels=labels, right=False)
df['Pre_qualification_group'] = df['Pre_qualification_group'].map({'0-100': 0, '101-150': 1,'151-200': 2})

addmission_grade = df['Admission grade']
bins = [0, 100, 150, 200]
labels = ['0-100', '101-150', '151-200']
df['Addmission_grade_group'] = pd.cut(addmission_grade, bins=bins, labels=labels, right=False)
df['Addmission_grade_group'] = df['Addmission_grade_group'].map({'0-100': 0, '101-150': 1,'151-200': 2})

ages = df['Age at enrollment']
bins = [0, 20, 30, 40, 50, 100]
labels = ['0-20', '21-30', '31-40', '41-50', '51-100']
df['Age_group'] = pd.cut(ages, bins=bins, labels=labels, right=False)
df['Age_group'] = df['Age_group'].map({'0-20': 0, '21-30': 1,'31-40': 2, '41-50': 3, '51-100': 4})

# Curricular units 1st sem (grade)
grade1 = df['Curricular units 1st sem (grade)']
bins = [0, 5, 10, 15, 20]
labels = ['0-5', '5-10', '10-15', '15-20']
df['grade1_group'] = pd.cut(grade1, bins=bins, labels=labels, right=False)
df['grade1_group'] = df['grade1_group'].map({'0-5': 0, '5-10': 1,'10-15': 2,'15-20': 3})

# Curricular units 2nd sem (grade)
grade2 = df['Curricular units 2nd sem (grade)']
bins = [0, 5, 10, 15, 20]
labels = ['0-5', '5-10', '10-15', '15-20']
df['grade2_group'] = pd.cut(grade2, bins=bins, labels=labels, right=False)
df['grade2_group'] = df['grade2_group'].map({'0-5': 0, '5-10': 1,'10-15': 2,'15-20': 3})

### Feature Selection

In [115]:
import scipy.stats as stats
from scipy.stats import chi2_contingency

# Done Chi_square_test to select only significant features
def chi_square_test(data, feature, target, significant_features):
  contingency_table = pd.crosstab(data[feature], data[target])
  observed = contingency_table.values
  col_totals = observed.sum(axis=0)
  row_totals = observed.sum(axis=1)
  expected = np.outer(row_totals, col_totals)/data.shape[0]

  epsilone = 1e-10
  chi_squared_stat = ((observed - expected) ** 2/ expected).sum().sum()
  degrees_of_freedom = (observed.shape[0] - 1) * (observed.shape[1] - 1) + epsilone
  chi_squared_stat /= degrees_of_freedom

  p_value = 1 - stats.chi2.cdf(chi_squared_stat, degrees_of_freedom)

  if p_value < 0.05:
    print(f'\n<(feature)>')
    print('Chi-square: []'.format(round(chi_squared_stat, 2)))
    print('P-value: {}'.format(round (p_value, 7)))
    significant_features.append(feature)

significant_features = []

for cols in df.columns[:-1]:
  chi_square_test(df, cols, 'Target', significant_features)



<(feature)>
Chi-square: []
P-value: 0.0007579

<(feature)>
Chi-square: []
P-value: 5e-07

<(feature)>
Chi-square: []
P-value: 0.0

<(feature)>
Chi-square: []
P-value: 0.0

<(feature)>
Chi-square: []
P-value: 0.0

<(feature)>
Chi-square: []
P-value: 0.0

<(feature)>
Chi-square: []
P-value: 0.0014575

<(feature)>
Chi-square: []
P-value: 0.0

<(feature)>
Chi-square: []
P-value: 1.9e-06

<(feature)>
Chi-square: []
P-value: 0.0


In [116]:
significant_features

['Daytime/evening attendance\t',
 'Displaced',
 'Debtor',
 'Tuition fees up to date',
 'Gender',
 'Scholarship holder',
 'Curricular units 2nd sem (approved)',
 'Target',
 'Age_group',
 'grade1_group']

In [117]:
# Since above significant_features includes 'Target',
# created the list of feature without 'Target'
names = ['Daytime/evening attendance\t','Displaced','Debtor',
         'Tuition fees up to date','Gender','Scholarship holder',
         'Curricular units 2nd sem (approved)','Age_group','grade1_group']

Utilizing only significant features offers benefits such as simplification and interpretability of models, reduction of uncertainty, better generalization, and potentially improved performance. By eliminating unnecessary information, models can make more informed decisions and produce more reliable predictions.

## Implement Decision Tree Model

In [118]:
class DecisionTree:
    def __init__(self, criterion='entropy', max_depth=5, min_samples=1): # Set default parameters
        self.tree = None
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples = min_samples

    def _entropy(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        entropy = -np.sum(probs * np.log2(probs))
        return entropy

    def _gini(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        gini = 1 - np.sum(probs ** 2)
        return gini

    def _split_dataset(self, X, y, feature_idx, threshold):
        left_mask = X[:, feature_idx] <= threshold
        right_mask = ~left_mask
        return X[left_mask], X[right_mask], y[left_mask], y[right_mask]

    def _calculate_split_criteria(self, y, y_left, y_right):
        if self.criterion == 'entropy':
            information_gain = self._entropy(y) - (len(y_left) / len(y)) * self._entropy(y_left) - (len(y_right) / len(y)) * self._entropy(y_right)
            return information_gain
        elif self.criterion == 'gain_ratio':
            information_gain = self._entropy(y) - (len(y_left) / len(y)) * self._entropy(y_left) - (len(y_right) / len(y)) * self._entropy(y_right)
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", category=RuntimeWarning)
                split_info = -((len(y_left) / len(y)) * np.log2(len(y_left) / len(y)) + (len(y_right) / len(y)) * np.log2(len(y_right) / len(y)))
            return information_gain / split_info if split_info != 0 else 0  # Avoid division by zero
        elif self.criterion == 'gini':
            gini_impurity = self._gini(y) - (len(y_left) / len(y)) * self._gini(y_left) - (len(y_right) / len(y)) * self._gini(y_right)
            return gini_impurity

    def _find_best_split(self, X, y):
        best_criteria = -1
        best_feature_idx = None
        best_threshold = None
        for feature_idx in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                X_left, X_right, y_left, y_right = self._split_dataset(X, y, feature_idx, threshold)
                criteria = self._calculate_split_criteria(y, y_left, y_right)
                if criteria > best_criteria:
                    best_criteria = criteria
                    best_feature_idx = feature_idx
                    best_threshold = threshold
        return best_feature_idx, best_threshold

    def _build_tree(self, X, y, depth=0):
        if (self.max_depth is not None and depth >= self.max_depth) or len(np.unique(y)) == 1 or len(y) < self.min_samples:
            return np.bincount(y).argmax(), None, None, None
        feature_idx, threshold = self._find_best_split(X, y)
        if feature_idx is None or threshold is None:
            return np.bincount(y).argmax(), None, None, None
        X_left, X_right, y_left, y_right = self._split_dataset(X, y, feature_idx, threshold)
        left_subtree = self._build_tree(X_left, y_left, depth + 1)
        right_subtree = self._build_tree(X_right, y_right, depth + 1)
        return (feature_idx, threshold, left_subtree, right_subtree)


    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _predict_instance(self, x, tree):
        if tree[1] is None:  # Leaf node
            return tree[0]
        feature_idx, threshold, left_subtree, right_subtree = tree
        if threshold is not None:  # Ensure threshold is not None
            if x[feature_idx] <= threshold:
                return self._predict_instance(x, left_subtree)
            else:
                return self._predict_instance(x, right_subtree)
        else:  # Handle case where threshold is None
            # If threshold is None, we cannot make a decision, so return a default value
            # You can decide what value to return here based on your application
            return -1  # For example, returning -1 indicates an unknown classification

    def predict(self, X):
        predictions = []
        for x in X:
            prediction = self._predict_instance(x, self.tree)
            predictions.append(prediction)
        return np.array(predictions)

# To calculate the accuracy score
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)



## Random Sampling

In [121]:
# Function to split data into train and test sets
def train_test_split(X, y, test_size=0.3): # Use 30% for testing
    num_samples = X.shape[0]
    num_test_samples = int(test_size * num_samples)

    test_indices = np.random.choice(num_samples, num_test_samples, replace=False)
    train_indices = np.setdiff1d(np.arange(num_samples), test_indices)

    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    return X_train, X_test, y_train, y_test

X = df[names].values # significant_features
y = df['Target'].values#.reshape(-1,1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

## View Accuracy

In [123]:
tree_entropy = DecisionTree(criterion='entropy', max_depth=5, min_samples=2)
tree_entropy.fit(X_train, y_train)
y_pred = tree_entropy.predict(X_test)
acc = accuracy(y_test, y_pred)
print(f"Accuracy using Entropy: {acc}")

Accuracy using Entropy: 0.7490580256217031


'\nfor criterion in criteria:\n    tree = DecisionTree(criterion)\n    tree.fit(X_train, y_train)\n    y_pred = tree.predict(X_test)\n    acc = accuracy(y_test, y_pred)\n    print(f"Accuracy using {criterion.capitalize()}: {acc}")\nprint()\n'

In [124]:
tree_gainRatio = DecisionTree(criterion='gain_ratio', max_depth=7, min_samples=4)
tree_gainRatio.fit(X_train, y_train)
y_pred = tree_gainRatio.predict(X_test)
acc = accuracy(y_test, y_pred)
print(f"Accuracy using Gain Ratio: {acc}")

Accuracy using Gain Ratio: 0.7445365486058779


In [131]:
tree_gini = DecisionTree(criterion='gini', max_depth=3, min_samples=2)
tree_gini.fit(X_train, y_train)
y_pred = tree_gini.predict(X_test)
acc = accuracy(y_test, y_pred)
print(f"Accuracy using Gini Impurity: {acc}")

Accuracy using Gini Impurity: 0.7430293896006028


### Summary:
- Accuracy using Entropy: 74.91%
- Accuracy using Gain Ratio: 74.45%
- Accuracy using Gini Impurity: 74.30%

### Why I used pre-pruning technique?
Using pre-pruning techniques, even in scenarios where overfitting may not be a significant concern, can still provide tangible benefits in terms of simplicity, efficiency, robustness, and future-proofing of your machine learning models.

## Conclusion

- The three criteria, Entropy, Gain Ratio, and Gini Impurity, showed very similar accuracies.
- The choice of hyperparameters also varied across the criteria, with different optimal values for max_depth and min_samples_split.


Further analysis and experimentation may be required to understand the performance differences across the criteria and fine-tune the model for better accuracy.

(If post-pruning had been available, better results could have been expected. However, since only pre-pruning technique was mentioned in the assignment documentation, it was not performed.)